In [36]:
# python packages pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import inspect

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Activation
from keras.layers import CuDNNLSTM
from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
from keras.layers import Dropout

from sklearn.model_selection import train_test_split
import importlib
import utilis

# custom
from keras import backend as K
from keras.layers import Layer
from keras.constraints import MinMaxNorm
from keras import initializers, regularizers, constraints, Input
from keras.models import Model

In [2]:
sys.path.append("..")

In [3]:
# custom python scripts
from packages import generator

In [4]:
importlib.reload(generator)

<module 'packages.generator' from '../packages/generator.py'>

In [5]:
# # check version
# print(inspect.getsource(generator.Keras_DataGenerator))

# Bidirectional LSTM with Hypotheses

In [6]:
# Check that you are running GPU's
utilis.GPU_checker()





You are runnning an instance with 4 GPU's


In [7]:
utilis.aws_setup()

AWS SETUP SHOULD BE COMPLETE, we are on <botocore.client.S3 object at 0x7f62b847a940>


# Config, generators and train

In [8]:
INPUT_TENSOR_NAME = "inputs_input"
SIGNATURE_NAME = "serving_default"
W_HYP = True
LEARNING_RATE = 0.001
BATCH_SIZE = 64

# constnats
VOCAB_SIZE = 1254
INPUT_LENGTH = 3000 if W_HYP else 1000
EMBEDDING_DIM = 512

print(INPUT_LENGTH)

3000


In [9]:
importlib.reload(generator)

# generators
training_generator = generator.Keras_DataGenerator(data_dir='', subset_frac=0.04,  dataset='train_new', w_hyp=W_HYP)
print()
validation_generator = generator.Keras_DataGenerator(data_dir='', subset_frac=0.04, dataset='valid_new', w_hyp=W_HYP)

# of batches:  5888.0
# of batches reduced to:  235.52
Generating examples from a set of 15073.28 examples 


# of batches:  1600.0
# of batches reduced to:  64.0
Generating examples from a set of 4096.0 examples 



In [40]:
# custom dot product function
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
# find a way to return attention weight vector a
class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        # initialization of all learnable params
        self.init = initializers.get('glorot_uniform')
        
        # regularizers for params, init as None
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        # constraints for params, init as None
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
#         assert len(input_shape) == 3
        
        # weight matrix
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        # bias term
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='lecun_uniform',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        
        # context vector
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)
        
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)
        
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
#         a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon() * 100, K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]  

    
# model
def build_model(vocab_size, embedding_dim, input_length):
    sequence_input = Input(shape=(input_length,), dtype='int32')
    embedded_sequences = Embedding(vocab_size, embedding_dim, input_length=input_length)(sequence_input)
    output_1 = SpatialDropout1D(0.2)(embedded_sequences)
    output_2 = Bidirectional(CuDNNLSTM(512, return_sequences=True, 
                                       kernel_constraint=MinMaxNorm(min_value=0.0001, max_value=1.0, rate=1.0, axis=0), 
                                       recurrent_constraint=MinMaxNorm(min_value=0.0001, max_value=1.0, rate=1.0, axis=0), 
                                       bias_constraint = MinMaxNorm(min_value=0.0001, max_value=1.0, rate=1.0, axis=0)))(output_1)
    context_vec = AttentionWithContext(
        W_constraint = MinMaxNorm(min_value=0.0001, max_value=1.0, rate=1.0, axis=0),
        u_constraint = MinMaxNorm(min_value=0.0001, max_value=1.0, rate=1.0, axis=0),
        b_constraint = MinMaxNorm(min_value=0.0001, max_value=1.0, rate=1.0, axis=0))(output_2)
    predictions = Dense(41, kernel_constraint = MinMaxNorm(min_value=0.0001, max_value=1.0, rate=1.0, axis=0), activation='softmax')(context_vec)
    model = Model(inputs=sequence_input, outputs=predictions)
    return model

### testing generator

In [41]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 3000)              0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 3000, 512)         642048    
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 3000, 512)         0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 3000, 1024)        4202496   
_________________________________________________________________
attention_with_context_10 (A (None, 1024)              1050624   
_________________________________________________________________
dense_8 (Dense)              (None, 41)                42025     
Total params: 5,937,193
Trainable params: 5,937,193
Non-trainable params: 0
_________________________________________________________________


In [42]:
## ARE YOU LOADING A MODEL IF YES RUN TEH FOLLOWING LINES 
# from keras.models import model_from_json
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
# # REMEMEBER TO COMPILE 
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
#overwriting model
# model = loaded_model

In [46]:
model.layers[4].get_weights()

[array([[-0.0217997 ,  0.03969083,  0.0215626 , ...,  0.03818964,
          0.01074576,  0.04020418],
        [-0.00333207, -0.05536939, -0.05589694, ...,  0.03243203,
          0.00509956, -0.04171598],
        [ 0.0343883 ,  0.04647451, -0.00663181, ..., -0.00550194,
         -0.00727929, -0.01725633],
        ...,
        [ 0.04436937, -0.02148931, -0.00088677, ..., -0.01644387,
          0.0431393 , -0.01350097],
        [ 0.02521509,  0.03857413,  0.021363  , ...,  0.03159701,
         -0.00033784, -0.03753368],
        [ 0.01121367,  0.00749657, -0.02630752, ...,  0.0053083 ,
          0.02673394, -0.03542515]], dtype=float32),
 array([ 0.04667596, -0.02666715, -0.03496671, ...,  0.04108815,
         0.03442511,  0.02041246], dtype=float32),
 array([-0.01997736, -0.02414181,  0.02603782, ...,  0.03218105,
         0.0073827 , -0.03639194], dtype=float32)]

In [47]:
%%time
#try and make it run until 9 am GMT+1
n_epochs = 2
history = model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,
                            verbose=1,
                            use_multiprocessing=True,
                            epochs=n_epochs)

Epoch 1/2
235/235 [==============================] - 635s 3s/step - loss: 2.6802 - acc: 0.1906 - val_loss: 2.6020 - val_acc: 0.2065
Epoch 2/2
235/235 [==============================] - 600s 3s/step - loss: 2.5519 - acc: 0.2275 - val_loss: 2.6120 - val_acc: 0.1992
CPU times: user 5min 14s, sys: 1min 34s, total: 6min 49s
Wall time: 20min 35s


## Save modek

In [ ]:
# FOR SAVING MODEL
model_json = model_GPU.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [83]:
#WARNING_DECIDE_HOW_TO_NAME_LOG
#descriptionofmodel_personwhostartsrun
#e.g. LSTM_128encoder_etc_tanc
LOSS_FILE_NAME = "forjeff2"

#WARNING NUMBER 2 - CURRENTLY EVERYTIME YOU RERUN THE CELLS BELOW THE FILES WITH THOSE NAMES GET WRITTEN OVER

In [81]:
# save history - WARNING FILE NAME 
utilis.history_saver_bad(history, LOSS_FILE_NAME)

SAVED SOME LOGS -- OVERWROTE OLD LOGS -- SOMEONE NEEDS TO FIX THIS
